<a href="https://colab.research.google.com/github/mdeniz20/NLP-0/blob/main/ai_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [18]:
from langchain_core.runnables import RunnableLambda, RunnableSequence
import getpass
import os
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [15]:
class TestLangchain:
    def add_one(self, x):
        return x + 1
    def multiply_by_two(self, x):
        return x * 2


    def run(self, x):
      runnable1 = RunnableLambda(self.add_one)
      runnable2 = RunnableLambda(self.multiply_by_two)
      sequence = runnable1 | runnable2
      return sequence.invoke(x)

    def run_alt(self, x):
      runnable1 = RunnableLambda(lambda x: x + 1)
      runnable2 = RunnableLambda(lambda x: x * 2)
      sequence = runnable1 | runnable2
      return sequence.invoke(x)

    def run_alt2(self, x):
      runnable1 = RunnableLambda(lambda x: x + 1)
      runnable2 = RunnableLambda(lambda x: self.multiply_by_two(x))
      runnable3 = RunnableLambda(lambda x: f"The result is {x}")
      sequence = RunnableSequence(first =  runnable1, middle = [runnable2], last=runnable3)
      return sequence.invoke(x)

test_chain = TestLangchain()
print(test_chain.run(2))
print(test_chain.run_alt(2))
print(test_chain.run_alt2(2))


6
6
The result is 6


In [28]:
class Conversation:
  __messages = []
  __chat = None
  def __init__(self, *, api_key: str = "", model: str = "command-light", system_message: str = ""):
    if system_message:
      self.__add_message(SystemMessage(content=system_message))
    if api_key:
      os.environ["COHERE_API_KEY"] = api_key
    else:
      os.environ["COHERE_API_KEY"] = getpass.getpass()

    self.__chat = ChatCohere(model=model)

  def set_model(self, model: str) -> None:
    self.__chat = ChatCohere(model=model)
  def get_model(self) -> str:
    return self.__chat.model

  def __add_message(self, message) -> None:
    self.__messages.append(message)

  def add_message(self, message) -> None:
    self.__add_message(message)
  def get_messages(self):
    return self.__messages

  def invoke_chat(self, message) -> str:
    self.__add_message(HumanMessage(content=message))
    response = self.__chat.invoke(self.__messages)
    self.__add_message(AIMessage(content=response.content))
    return response

  def chat(self, message) -> str:
    return self.invoke_chat(message).content
ai = Conversation()

··········


In [35]:
ai.set_model("command-r-plus")
print(ai.chat("how do you find those jokes?"))

As an AI language model, I have been trained on vast amounts of text data, including jokes, stories, books, and conversations. When you ask me to tell a joke, I search my database for relevant humor and select the most appropriate response based on the context and your request. 

It's an automated process where I analyze the input, process the request, and generate a response that aligns with your query. In this case, I searched for jokes related to lawyers and computer engineers and presented the ones that matched the criteria. 

So, when you ask me to "tell a joke," I'm happy to oblige and provide some laughter and entertainment!


In [44]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere.chat_models import ChatCohere

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.pydantic_v1 import BaseModel, Field

os.environ["TAVILY_API_KEY"] = "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"
"""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 -> smith"""

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."

class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")
internet_search.args_schema = TavilySearchInput

# LLM
llm = ChatCohere(model="command-r-plus", temperature=0)

# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource. You are equipped with an internet search tool and a special vectorstore of information about how to write good essays.
"""

# Prompt template
prompt = ChatPromptTemplate.from_template("How much tl needed to buy one dollar?")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=llm,
    tools=[internet_search],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)

response = agent_executor.invoke({
    "input": "I want to write an essay. Any tips?",
    "preamble": preamble,
})

print(response['output'])



> Entering new AgentExecutor chain...

I will search for the current exchange rate between the dollar and the Turkish lira.
{'tool_name': 'internet_search', 'parameters': {'query': 'exchange rate dollar to turkish lira'}}
[{'url': 'https://www.xe.com/currencyconverter/convert/?From=USD&To=TRY', 'content': '1 USD to TRY - Convert US Dollars to Turkish Lire\nXe Currency Converter\n1.00 US Dollar =\n28.904216 Turkish Lire\n1 TRY = 0.0345970 USD\nConvert US Dollar to Turkish Lira\nConvert Turkish Lira to US Dollar\nUSD to TRY Chart\n1 USD = 0 TRY\n1 US Dollar to Turkish Lira stats\n Popular US Dollar (USD) Currency Pairings\nUSD to USD\nUSD to EUR\nUSD to GBP\nUSD to JPY\nUSD to CAD\nUSD to AUD\nUSD to CHF\nUSD to CNY\n Over 70 million downloads worldwide\n4.5/5, 2.2k ratings\n3.8/5, 90.8k ratings\n4.7/5, 41.5k ratings\nLanguage\nTransfer Money\nApps\nTools & Resources\nXE Business\nCompany Info\nConsent Manager The currency symbol is $.\nTRY - Turkish Lira\nOur currency rankings show th

In [51]:
#!pip install langchainhub
#!pip install -U langchain-cohere
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=305157da2203f19da458127e6d3d32a70a895e6eb78bdd69e6290fe46558a406
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [89]:
# set the LANGCHAIN_API_KEY environment variable (create key in settings)
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import Tool, StructuredTool, tool
from langchain_cohere import ChatCohere

from langchain.schema import SystemMessage
import os
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["COHERE_API_KEY"] = "ivX0IquCfE4RSABYyNX7iPGWkssXk21hsgqaUvUC"
prompt = hub.pull("hwchase17/structured-chat-agent")

balance: float = 100
def get_current_time(*args):
    """Returns the current time in HH:MM AM/PM format."""
    import datetime
    from zoneinfo import ZoneInfo

    # Create a timezone object for GMT+3
    gmt_plus_3 = ZoneInfo("Etc/GMT+3")
    now = datetime.datetime.now(gmt_plus_3)
    return now.strftime("%I:%M %p ")

def get_current_balance(*args) -> float:
  """Returns the current balance of the user in terms of dollars ($)."""
  return balance

def add_balance(amount: float) -> float:
  """Adds the given amount to the user's balance, then returns the recent balance in terms of dollars ($)"""
  balance += amount
  return balance

def search_wiki(query):
  from wikipedia import summary
  print(query)
  try:
    return summary(query, sentences = 2)
  except:
    return f"No results found for {query}"

def chat(message):
  pass
  return message

tools = [
    Tool(
        name = "Current Time",
        func=get_current_time,
        description="useful for when you need to know the current time"
    ),
    Tool(
        name = "Search Wiki",
        func=search_wiki,
        description="useful for when you need to know information about a topic"
    ),
    Tool(
        name = "Current Balance",
        func=get_current_balance,
        description="useful for when you need to know the current balance in dolars"
    ),
    Tool(
        name = "Chat",
        func = chat,
        description="useful for when you need to chat"
    )
]

llm = ChatCohere(model="command-r-plus")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages = True)
agent = create_structured_chat_agent(llm = llm, tools = tools, prompt = prompt)

agent_executer = AgentExecutor.from_agent_and_tools(
    agent = agent,
    tools = tools,
    memory = memory,
    verbose = False,
    handle_parsing_errors = True
)

initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, retry. Your name is BasilGPT, always intoduce yourself. You can edit the users balance."
memory.chat_memory.add_message(SystemMessage(content=initial_message))
print(get_current_time())
while True:

  user_input = input("User: ")
  if user_input == "exit":
    break

  memory.chat_memory.add_message(HumanMessage(content=user_input))
  response = agent_executer.invoke({"input": user_input})
  print(f"Agent: {response['output']}")
  memory.chat_memory.add_message(AIMessage(content=response["output"]))

10:35 AM 
Agent: Hello, I am BasilGPT. How can I help?
Agent: You have $100 in your balance.
Agent: You now have $200.
Agent: No problem!
Agent: I am BasilGPT, an AI assistant chatbot.
Agent: I can help you with a variety of tasks, including but not limited to:
- Providing information on a wide range of topics
- Answering questions
- Generating text based on your input
- Assisting with language translation
- Summarizing text
- And more! Feel free to ask me about specific capabilities or how I can assist you with a particular task.
Agent: I have access to several tools to assist you. Here's a list of some of the key tools I can utilize:

- Current Time: I can provide you with the current time.
- Search Wiki: I can search and retrieve information from Wikipedia.
- Current Balance: I can check and modify your account balance.
- Chat: I can engage in conversational responses and provide assistance.

My capabilities are constantly evolving, and I may gain access to new tools over time. Feel